# Libraries

In [1]:
using QuantumOptics
using LinearAlgebra
using SparseArrays

# Fock Space

In [2]:
Nx = Ny = 6
N = Nx*Ny

36

In [3]:
max_occupation_number = 3
occ_array = []
for i in 0:max_occupation_number
    push!(occ_array, i)
end

In [4]:
b = NLevelBasis(N)
states = bosonstates(b, [occ_array...])
b_mb = ManyBodyBasis(b, states)

ManyBody(onebodybasis=NLevel(N=36), states:9139)

# PBC on Lattice

In [5]:
using OffsetArrays

In [6]:
lat = range(1,Nx*Ny)
latt = reshape(lat, (Nx,Ny))
lattice = OffsetArray(latt, 0:Nx-1, 0:Ny-1)
x_co = range(0, Nx-1) 
y_co = range(0, Ny-1)
arr = []
for i in 1:length(x_co)
    for j in 1:length(y_co)
       arr = [arr;x_co[i];y_co[j]]
    end
end
xy = reshape(arr, (2,Nx*Ny)) |> transpose
arr = OffsetArray(arr,0:Nx*Ny*2-1)
lattice

6×6 OffsetArray(reshape(::UnitRange{Int64}, 6, 6), 0:5, 0:5) with eltype Int64 with indices 0:5×0:5:
 1   7  13  19  25  31
 2   8  14  20  26  32
 3   9  15  21  27  33
 4  10  16  22  28  34
 5  11  17  23  29  35
 6  12  18  24  30  36

In [7]:
x = []
function PerBC()
    for i in 0:Nx-1
        for j in 0:Ny-1        
            cc = [lattice[mod(j,Ny),mod(i-1,Nx)],lattice[mod(j+1,Ny),mod(i,Nx)],lattice[mod(j,Ny),mod(i+1,Nx)],lattice[mod(j-1,Ny),mod(i,Nx)]]
            push!(x,cc)
        end
    end
    return x
end
for i in 1:N
    println("$i: ", PerBC()[i])
end

1: [31, 2, 7, 6]
2: [32, 3, 8, 1]
3: [33, 4, 9, 2]
4: [34, 5, 10, 3]
5: [35, 6, 11, 4]
6: [36, 1, 12, 5]
7: [1, 8, 13, 12]
8: [2, 9, 14, 7]
9: [3, 10, 15, 8]
10: [4, 11, 16, 9]
11: [5, 12, 17, 10]
12: [6, 7, 18, 11]
13: [7, 14, 19, 18]
14: [8, 15, 20, 13]
15: [9, 16, 21, 14]
16: [10, 17, 22, 15]
17: [11, 18, 23, 16]
18: [12, 13, 24, 17]
19: [13, 20, 25, 24]
20: [14, 21, 26, 19]
21: [15, 22, 27, 20]
22: [16, 23, 28, 21]
23: [17, 24, 29, 22]
24: [18, 19, 30, 23]
25: [19, 26, 31, 30]
26: [20, 27, 32, 25]
27: [21, 28, 33, 26]
28: [22, 29, 34, 27]
29: [23, 30, 35, 28]
30: [24, 25, 36, 29]
31: [25, 32, 1, 36]
32: [26, 33, 2, 31]
33: [27, 34, 3, 32]
34: [28, 35, 4, 33]
35: [29, 36, 5, 34]
36: [30, 31, 6, 35]


# Hopping Phases

In [8]:
function HP(m, n, alpha)
    if abs(xy[m,1]-xy[n,1])==Nx-1
        if xy[m,1] > xy[n,1]
            P = exp(-1im*2*pi*alpha*xy[m,2])
        elseif xy[m,1] < xy[n,1]
            P = exp(1im*2*pi*alpha*xy[m,2])
        end
    elseif m==n
        P = 0
    else
        if xy[m,1] > xy[n,1]
            P = exp(1im*2*pi*alpha*xy[m,2])
        elseif xy[m,1] < xy[n,1]
            P = exp(-1im*2*pi*alpha*xy[m,2])
        else
            P = exp(0)
        end
    end
end 

HP (generic function with 1 method)

# Hofstadter-Hubbard Hamiltonian

\begin{align}
&N/N_{\alpha}=1/2 \\
&N_{\alpha}=N_xN_y\alpha
\end{align}

In [9]:
4/(8*8*1/8) #Topological Degeneracy

0.5

In [10]:
alpha = 1/6
t = 1
U = 2

2

In [ ]:
#kinetic term
KT = SparseOperator(b_mb)
#interaction term
IT = SparseOperator(b_mb)
for m in 1:N
    IT = IT + U/2 * number(b_mb, m) * (number(b_mb, m) - identityoperator(b_mb))
    for n in 1:N
        if m in PerBC()[n]
            KT = KT - t * HP(m, n, alpha) * transition(b_mb, m, n)
        end
    end
end
HH = KT + IT
eigenenergies(dense(HH))[1]

###### Laughlin States

In [ ]:
using Plots
eig = eigenenergies(dense(HH))
x=1:length(eig)
gr()
plot(x, eig, seriestype = :scatter, xlabel="Öz-Değer Sırası", ylabel="Enerjiler", markersize = 5, label="U=2", legend = :outertopleft)

# Calculate The Expected Values

In [24]:
A = eigenstates(dense(HH))

([-2.6936047825375495, -1.9021130325903024, -0.8744137645212482, -0.6180339887498945, -0.00635110067271949, -0.0016488562721854727, 0.0, 0.6180339887498965, 0.8736498324987707, 1.902113032590307, 2.6863687145603, 1000.0007639320219, 1000.0016488562711, 1000.0063511006717, 1000.0072360679766], Ket{ManyBodyBasis{Vector{Int64}, NLevelBasis{Int64}, 0xd7173e7347b478b9, UInt64}, Vector{ComplexF64}}[Ket(dim=15)
  basis: ManyBody(onebodybasis=NLevel(N=4), states:15)
                  0.0 + 0.0im
                  0.0 + 0.0im
                  0.0 + 0.0im
                  0.0 + 0.0im
                  0.0 + 0.0im
 -0.00029329910957497 - 0.00090268184100707im
             2.08e-15 - 0.3537897988901847im
  -0.2079524261991705 - 0.2862219597386506im
  -0.4042363774833981 - 0.2936949196085103im
  0.00029329910957479 - 0.00090268184100693im
 -0.15440455670951397 - 0.47520836224972873im
 -0.20795242619917395 - 0.2862219597386485im
 -0.00076786703773712 - 0.00055788805876974im
  -0.3364740936332595 -

Örgüdeki parçacık sayı dağılımlarına karşılık gelen enerjiler

In [25]:
C = Array{Float64}(undef, length(states), 2)
for i in 1:length(states)
    exp = round(expect(number(b_mb), A[2][i]))
    C[i] = exp 
    C[i,2] = A[1][i]
end
C
CC = sortslices(C, dims=1 ,by = x -> x[1])
#show(stdout, "text/plain", CC)

15×2 Matrix{Float64}:
 0.0     0.0
 1.0    -1.90211
 1.0    -0.618034
 1.0     0.618034
 1.0     1.90211
 2.0    -2.6936
 2.0    -0.874414
 2.0    -0.0063511
 2.0    -0.00164886
 2.0     0.87365
 2.0     2.68637
 2.0  1000.0
 2.0  1000.0
 2.0  1000.01
 2.0  1000.01

###### Data frame from Matrix

In [13]:
using DataFrames

In [14]:
df = DataFrame(CC, :auto)
rename!(df,:x1 => :Parçacık_Sayısı)
rename!(df,:x2 => :Enerji)

,Parçacık_Sayısı,Enerji
,Float64,Float64
1,0.0,0.0
2,1.0,-1.90211
3,1.0,-0.618034
4,1.0,0.618034
5,1.0,1.90211
6,2.0,-2.6936
7,2.0,-0.874414
8,2.0,-0.0063511
9,2.0,-0.00164886


# Hard-Core Interaction

Örgüde en fazla 2 parçacık varken, her sitede en fazla 1 parçacık bulunma etkileşimi

In [14]:
b_hard = NLevelBasis(N)
states_hard = fermionstates(b_hard, [2])
b_mb_hard = ManyBodyBasis(b_hard, states_hard)
for i in 1:length(states_hard)
    println("$i: ", states_hard[i])
end

1: [1, 1, 0, 0]
2: [1, 0, 1, 0]
3: [1, 0, 0, 1]
4: [0, 1, 1, 0]
5: [0, 1, 0, 1]
6: [0, 0, 1, 1]


In [15]:
UU=2
#kinetic term
KT_hard = SparseOperator(b_mb_hard)
#interaction term
IT_hard = SparseOperator(b_mb_hard)
for m in 1:N
    IT_hard = IT_hard + UU/2 * number(b_mb_hard, m) * (number(b_mb_hard, m) - identityoperator(b_mb_hard))
    for n in 1:N
        if m in PerBC()[n]
            KT_hard = KT_hard - t * HP(m, n, alpha) * transition(b_mb_hard, m, n)
        end
    end
end
HH_hard = KT_hard + IT_hard
eigenenergies(dense(HH_hard))
#Bu enerji seviyeleri, U çok büyükken, yukarıdaki Hamiltonyende (parçacık=2 ve oc. num=2) iki parçacıklı durumların enerjilerine denk düşüyor.

6-element Vector{Float64}:
 -2.6899940478558286
 -0.8740320488976425
 -2.0339045907339023e-16
  1.2839680890234273e-16
  0.8740320488976425
  2.6899940478558286